# Solution

## Data loading

In [75]:
import pandas as pd

pd.options.display.max_columns = 100

dtypes = {
    'delivery_type': 'category',
    'addr_region_reg': 'category',
    'addr_region_fact': 'category',
    'channel_name': 'category',
    'channel_name_2': 'category',
    'sas_limit_after_003_amt': 'uint8',
    'sas_limit_last_amt': 'uint8',
    'channel_name_modified_2018': 'category',
    'clnt_education_name': 'category',
    'clnt_marital_status_name': 'category',
    'clnt_employment_type_name': 'category',
    'clnt_speciality_sphere_name': 'category',
    'clnt_sex_name': 'category',
    'prt_name': 'category',
    'clnt_income_month_avg_net_amt': 'uint',
    'clnt_expense_month_avg_amt': 'uint',
    'app_addr_region_reg': 'category',
    'app_addr_region_fact': 'category',
    'app_addr_region_sale': 'category',
    'prt_name': 'category',
    'feature_0': 'category',
    'clnt_birth_year': 'uint',
    'addr_region_fact_encoding1': 'float32',
    'addr_region_fact_encoding2': 'float32',
    'addr_region_reg_encoding1': 'float32',
    'addr_region_reg_encoding2': 'float32',
    'app_addr_region_reg_encoding1': 'float32',
    'app_addr_region_reg_encoding2': 'float32',
    'app_addr_region_fact_encoding1': 'float32',
    'app_addr_region_fact_encoding2': 'float32',
    'app_addr_region_sale_encoding1': 'float32',
    'app_addr_region_sale_encoding2': 'float32',
    'loans_main_borrower': 'float32',
    'loans_active': 'float32',
    'max_overdue_status': 'category',
    'ttl_inquiries': 'uint',
    'ttl_auto_loan': 'bool',
    'ttl_mortgage': 'bool',
    'ttl_credit_card': 'bool',
    'ttl_consumer': 'bool',
    'worst_status_ever': 'category',
    'feature_0': 'category',
    'feature_1': 'uint',
    'feature_2': 'uint',
    'feature_3': 'uint',
    'feature_4': 'uint',
    'feature_5': 'uint',
    'feature_6': 'uint',
    'feature_7': 'uint',
    'feature_8': 'uint',
    'feature_9': 'uint',
    'fl_coborrower': 'bool',
    'fl_active_coborrower': 'bool',
    'pay_load': 'float32',
    'makro_region': 'category',
    'fo': 'category',
    'region': 'category',
    'feature_30': 'category'
}

fill_missing = [
    ('inquiry_recent_period', 0, 'uint'),
    ('inquiry_1_week', 0, 'uint'),
    ('clnt_experience_cur_mnth', 0, 'uint'),
    ('clnt_experience_cur_year', 0, 'uint'),
    ('clnt_experience_total_mnth', 0, 'uint'),
    ('last_loan_date', 6200, 'uint'),
    ('first_loan_date', 6200, 'uint'),
    ('ttl_officials', -1, 'category'),
    ('ttl_legals', -1, 'category'),
    ('ttl_bankruptcies', -1, 'category'),
    ('inquiry_recent_period', 0, 'uint'),
    ('inquiry_3_month', 0, 'uint'),
    ('inquiry_6_month', 0, 'uint'),
    ('inquiry_9_month', 0, 'uint'),
    ('inquiry_12_month', 0, 'uint'),
    ('inquiry_1_week', 0, 'uint'),
    ('inquiry_1_month', 0, 'uint'),
    *[
        (f'feature_{i}', -1, 'float32')
        for i in range(10, 30)
        if i != 11
    ]
]

cols_to_use = (
    list(dtypes.keys()) +
    list(col for col, *_ in fill_missing) + 
    ['card_id', 'target']
)

df = pd.read_csv('data/train.csv', index_col='card_id', dtype=dtypes, usecols=cols_to_use)

for col, fill, dtype in fill_missing:
    df[col] = df[col].fillna(fill).astype(dtype)

df.head()

,target,delivery_type,addr_region_reg,addr_region_fact,channel_name,channel_name_2,channel_name_modified_2018,sas_limit_after_003_amt,sas_limit_last_amt,clnt_education_name,clnt_marital_status_name,clnt_employment_type_name,clnt_speciality_sphere_name,clnt_income_month_avg_net_amt,clnt_expense_month_avg_amt,clnt_experience_cur_mnth,clnt_experience_cur_year,clnt_experience_total_mnth,clnt_sex_name,app_addr_region_reg,app_addr_region_fact,app_addr_region_sale,prt_name,feature_0,clnt_birth_year,addr_region_fact_encoding1,addr_region_fact_encoding2,addr_region_reg_encoding1,addr_region_reg_encoding2,app_addr_region_reg_encoding1,app_addr_region_reg_encoding2,app_addr_region_fact_encoding1,app_addr_region_fact_encoding2,app_addr_region_sale_encoding1,app_addr_region_sale_encoding2,loans_main_borrower,loans_active,last_loan_date,first_loan_date,max_overdue_status,ttl_officials,ttl_legals,ttl_bankruptcies,inquiry_recent_period,inquiry_3_month,inquiry_6_month,inquiry_9_month,inquiry_12_month,ttl_inquiries,ttl_auto_loan,ttl_mortgage,ttl_credit_card,ttl_consumer,worst_status_ever,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,fl_coborrower,fl_active_coborrower,pay_load,inquiry_1_week,inquiry_1_month,feature_10,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,makro_region,fo,region,feature_30
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cid_10620,1,cat_1,107,107,cat_0,cat_3,cat_0,1,1,cat_1,cat_4,cat_0,cat_0,3,0,0,0,0,cat_0,107,107,45,cat_9,cat_7,46,6746.987793,7159.090820,6746.987793,7159.090820,6746.987793,7159.090820,6746.987793,7159.090820,6666.666504,6571.428711,0.88,0.78,193,2851,2,0.0,0.0,0.0,0,7,10,12,19,87,False,False,True,True,3,3,10,4,0,0,5,1,0,0,False,False,0.82,1,3,0.57,0.64,0.7,-1.0,-1.0,0.00,0.00,0.57,0.88,-1.0,0.00,0.77,0.49,0.53,0.91,0.99,-1.00,510.023529,-1.0,0,0,0,3
cid_105724,0,cat_1,9,9,cat_2,cat_5,cat_2,3,3,cat_1,cat_2,cat_3,cat_26,5,1,2,0,3,cat_1,9,9,1,cat_8,cat_9,47,7469.879395,7500.000000,7469.879395,7500.000000,7469.879395,7500.000000,7469.879395,7500.000000,7948.717773,7714.285645,0.77,0.70,73,4288,1,0.0,0.0,0.0,58,19,28,31,43,94,True,False,True,True,2,2,5,0,0,0,0,0,0,0,False,False,0.93,0,0,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.000000,-1.0,0,0,0,7
cid_101410,1,cat_1,109,109,cat_0,cat_3,cat_0,1,1,cat_1,cat_2,cat_3,cat_15,2,0,0,0,0,cat_0,109,109,109,cat_9,cat_10,44,7831.325195,8068.181641,7831.325195,8068.181641,7831.325195,8068.181641,7831.325195,8068.181641,5384.615234,5142.856934,0.51,0.00,242,1852,1,0.0,0.0,0.0,30,6,6,11,20,31,False,False,True,True,2,1,2,0,0,0,0,0,0,0,False,False,0.00,0,2,0.53,0.50,0.6,-1.0,-1.0,0.34,0.56,0.53,0.87,0.6,0.85,0.75,0.77,0.70,0.89,0.99,0.97,262.654236,359.0,2,4,71,4
cid_38961,0,cat_1,66,66,cat_0,cat_3,cat_0,3,3,cat_1,cat_2,cat_3,cat_14,2,1,0,0,0,cat_0,66,66,66,cat_9,cat_7,27,6746.987793,6704.545410,6746.987793,6704.545410,6746.987793,6704.545410,6746.987793,6704.545410,3076.923096,1714.285767,0.28,0.57,868,868,1,0.0,0.0,0.0,366,0,0,0,0,8,False,True,False,False,2,0,4,0,0,0,3,0,0,0,True,True,0.92,0,0,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.000000,-1.0,0,0,0,7
cid_57462,0,cat_1,16,16,cat_0,cat_3,cat_0,0,0,cat_4,cat_2,cat_3,cat_24,3,0,0,0,0,cat_0,16,16,16,cat_9,cat_13,42,7228.915527,7272.727051,7228.915527,7272.727051,7228.915527,7272.727051,7228.915527,7272.727051,4102.563965,3142.857178,0.28,0.00,1525,1525,1,0.0,0.0,0.0,26,4,7,8,15,33,False,False,True,False,1,0,0,0,0,0,0,0,0,0,False,False,0.00,0,3,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.000000,-1.0,0,0,0,7


## Feature extraction

In [76]:
class MeanEncoder:
    
    def __init__(self, df, on, by, prior_count=100):
        self.on = on
        self.by = by
        counts = df.groupby(by)[on].agg(['mean', 'count'])
        avg = df[on].mean()
        self.means = (
            counts
            .eval('(mean * count + @avg * @prior_count) / (count + @prior_count)')
            .rename(str(self))
        )
    
    def __str__(self):
        return f'avg_{self.on}_by_{self.by}'
        
    def transform(self, df):
        return df.join(self.means, on=self.by)[self.means.name]

In [77]:
class NMissing:
    
    def __str__(self):
        return 'n_missing'
    
    def transform(self, df):
        return df.isnull().sum(axis='columns').rename(str(self))

In [78]:
extractors = [
    MeanEncoder(df, on='target', by='addr_region_fact'),
    NMissing()
]

In [79]:
features = []

for ex in extractors:
    print(str(ex))
    features.append(ex.transform(df)) 
    
train = pd.concat([df] + features, axis='columns')
train.head()

avg_target_by_addr_region_fact
n_missing


,target,delivery_type,addr_region_reg,addr_region_fact,channel_name,channel_name_2,channel_name_modified_2018,sas_limit_after_003_amt,sas_limit_last_amt,clnt_education_name,clnt_marital_status_name,clnt_employment_type_name,clnt_speciality_sphere_name,clnt_income_month_avg_net_amt,clnt_expense_month_avg_amt,clnt_experience_cur_mnth,clnt_experience_cur_year,clnt_experience_total_mnth,clnt_sex_name,app_addr_region_reg,app_addr_region_fact,app_addr_region_sale,prt_name,feature_0,clnt_birth_year,addr_region_fact_encoding1,addr_region_fact_encoding2,addr_region_reg_encoding1,addr_region_reg_encoding2,app_addr_region_reg_encoding1,app_addr_region_reg_encoding2,app_addr_region_fact_encoding1,app_addr_region_fact_encoding2,app_addr_region_sale_encoding1,app_addr_region_sale_encoding2,loans_main_borrower,loans_active,last_loan_date,first_loan_date,max_overdue_status,ttl_officials,ttl_legals,ttl_bankruptcies,inquiry_recent_period,inquiry_3_month,inquiry_6_month,inquiry_9_month,inquiry_12_month,ttl_inquiries,ttl_auto_loan,ttl_mortgage,ttl_credit_card,ttl_consumer,worst_status_ever,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,fl_coborrower,fl_active_coborrower,pay_load,inquiry_1_week,inquiry_1_month,feature_10,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,makro_region,fo,region,feature_30,avg_target_by_addr_region_fact,n_missing
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cid_10620,1,cat_1,107,107,cat_0,cat_3,cat_0,1,1,cat_1,cat_4,cat_0,cat_0,3,0,0,0,0,cat_0,107,107,45,cat_9,cat_7,46,6746.987793,7159.090820,6746.987793,7159.090820,6746.987793,7159.090820,6746.987793,7159.090820,6666.666504,6571.428711,0.88,0.78,193,2851,2,0.0,0.0,0.0,0,7,10,12,19,87,False,False,True,True,3,3,10,4,0,0,5,1,0,0,False,False,0.82,1,3,0.57,0.64,0.7,-1.0,-1.0,0.00,0.00,0.57,0.88,-1.0,0.00,0.77,0.49,0.53,0.91,0.99,-1.00,510.023529,-1.0,0,0,0,3,0.662549,0
cid_105724,0,cat_1,9,9,cat_2,cat_5,cat_2,3,3,cat_1,cat_2,cat_3,cat_26,5,1,2,0,3,cat_1,9,9,1,cat_8,cat_9,47,7469.879395,7500.000000,7469.879395,7500.000000,7469.879395,7500.000000,7469.879395,7500.000000,7948.717773,7714.285645,0.77,0.70,73,4288,1,0.0,0.0,0.0,58,19,28,31,43,94,True,False,True,True,2,2,5,0,0,0,0,0,0,0,False,False,0.93,0,0,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.000000,-1.0,0,0,0,7,0.498339,0
cid_101410,1,cat_1,109,109,cat_0,cat_3,cat_0,1,1,cat_1,cat_2,cat_3,cat_15,2,0,0,0,0,cat_0,109,109,109,cat_9,cat_10,44,7831.325195,8068.181641,7831.325195,8068.181641,7831.325195,8068.181641,7831.325195,8068.181641,5384.615234,5142.856934,0.51,0.00,242,1852,1,0.0,0.0,0.0,30,6,6,11,20,31,False,False,True,True,2,1,2,0,0,0,0,0,0,0,False,False,0.00,0,2,0.53,0.50,0.6,-1.0,-1.0,0.34,0.56,0.53,0.87,0.6,0.85,0.75,0.77,0.70,0.89,0.99,0.97,262.654236,359.0,2,4,71,4,0.434518,0
cid_38961,0,cat_1,66,66,cat_0,cat_3,cat_0,3,3,cat_1,cat_2,cat_3,cat_14,2,1,0,0,0,cat_0,66,66,66,cat_9,cat_7,27,6746.987793,6704.545410,6746.987793,6704.545410,6746.987793,6704.545410,6746.987793,6704.545410,3076.923096,1714.285767,0.28,0.57,868,868,1,0.0,0.0,0.0,366,0,0,0,0,8,False,True,False,False,2,0,4,0,0,0,3,0,0,0,True,True,0.92,0,0,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.000000,-1.0,0,0,0,7,0.478198,0
cid_57462,0,cat_1,16,16,cat_0,cat_3,cat_0,0,0,cat_4,cat_2,cat_3,cat_24,3,0,0,0,0,cat_0,16,16,16,cat_9,cat_13,42,7228.915527,7272.727051,7228.915527,7272.727051,7228.915527,7272.727051,7228.915527,7272.727051,4102.563965,3142.857178,0.28,0.00,1525,1525,1,0.0,0.0,0.0,26,4,7,8,15,33,False,False,True,False,1,0,0,0,0,0,0,0,0,0,False,False,0.00,0,3,-1.00,-1.00,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.000000,-1.0,0,0,0,7,0.525827,0


## Learning

Separate features from target.

In [80]:
X = train.drop(columns='target')
y = train['target']

Train/test split.

In [81]:
from sklearn import model_selection

X_fit, X_val, y_fit, y_val = model_selection.train_test_split(
    X, y,
    random_state=42
)

LightGBM dance.

In [91]:
import lightgbm as lgb

fit = lgb.Dataset(X_fit, y_fit)
val = lgb.Dataset(X_val, y_val, reference=fit)

model = lgb.train(
    params={
        'learning_rate': .01,
        'objective': 'huber',
        'metric': 'auc',
        'boost_from_average': True,
        'min_data_in_leaf': 20,
        'max_bin': 255,
        'num_leaves': 31,
        'min_data_per_group': 100,
        'cat_smooth': 10,
        'seed': 42
    },
    train_set=fit,
    num_boost_round=10_000,
    valid_sets=[fit, val],
    valid_names=['fit', 'val'],
    early_stopping_rounds=20,
    verbose_eval=100
)

Training until validation scores don't improve for 20 rounds
[100]	fit's auc: 0.742577	val's auc: 0.72474
[200]	fit's auc: 0.754827	val's auc: 0.732144
[300]	fit's auc: 0.765007	val's auc: 0.736428
[400]	fit's auc: 0.773262	val's auc: 0.738302
[500]	fit's auc: 0.780281	val's auc: 0.739147
[600]	fit's auc: 0.786224	val's auc: 0.739599
Early stopping, best iteration is:
[617]	fit's auc: 0.787154	val's auc: 0.73967


In [ ]:
Training until validation scores don't improve for 20 rounds
[100]	fit's auc: 0.742629	val's auc: 0.724804
[200]	fit's auc: 0.753653	val's auc: 0.732281
[300]	fit's auc: 0.763307	val's auc: 0.736575
[400]	fit's auc: 0.771544	val's auc: 0.738544
[500]	fit's auc: 0.778577	val's auc: 0.739294
[600]	fit's auc: 0.784657	val's auc: 0.739872
[700]	fit's auc: 0.789985	val's auc: 0.740225
Early stopping, best iteration is:
[743]	fit's auc: 0.79211	val's auc: 0.740322


In [92]:
import numpy as np
from sklearn import metrics

y_pred = model.predict(X_val)

def roc_auc_score_at_K(predicted_proba, target, rate=0.1):  
    order = np.argsort(-predicted_proba) 
    top_k = int(rate * len(predicted_proba)) 
    return metrics.roc_auc_score(target[order][:top_k], predicted_proba[order][:top_k]) 

val_score = roc_auc_score_at_K(y_pred, y_val)
val_score

0.573095990189156

In [ ]:
0.5792567261842609

In [84]:
importances = model.feature_importance(importance_type='gain').astype(int)
print(pd.Series(importances, index=X_fit.columns).sort_values(ascending=False))

inquiry_recent_period            147412
prt_name                         124927
inquiry_1_week                    78754
addr_region_fact                  73187
addr_region_reg                   57963
                                  ...  
clnt_experience_cur_year              0
app_addr_region_reg_encoding2         0
ttl_legals                            0
ttl_bankruptcies                      0
n_missing                             0
Length: 92, dtype: int64


In [85]:
#model.save_model(f'models/model_{val_score:.4f}.lgb')
model.save_model(f'track_2/model.lgb')

In [86]:
import joblib

!rm -f track_2/*.pkl

for ex in extractors:
    joblib.dump(ex, f'track_2/{ex}.pkl')

## Custom loss

In [45]:
import lightgbm
import numpy as np
from scipy import special

def logloss_init_score(y):
    p = y.mean()
    p = np.clip(p, 1e-15, 1 - 1e-15)
    log_odds = np.log(p / (1 - p))
    return log_odds

def logloss_objective(preds, train_data):
    y = train_data.get_label()
    p = special.expit(preds)
    grad = p - y
    hess = p * (1 - p)
    
    #big = p > .6
    #grad[~big] = 0.1
    #hess[~big] = 0.1
    w = 5 ** (1 + p)
    return grad * w, hess * w
    
    #return grad, hess

def logloss_metric(preds, train_data):
    y = train_data.get_label()
    p = special.expit(preds)
    is_higher_better = True
    return 'auc', metrics.roc_auc_score(y, p), is_higher_better

fit = lightgbm.Dataset(
    X_fit, y_fit,
    init_score=np.full_like(y_fit, logloss_init_score(y_fit), dtype=float)
)

val = lightgbm.Dataset(
    X_val, y_val,
    init_score=np.full_like(y_val, logloss_init_score(y_fit), dtype=float),
    reference=fit
)

model = lightgbm.train(
    params={'learning_rate': 0.01},
    train_set=fit,
    num_boost_round=10000,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
    early_stopping_rounds=20,
    verbose_eval=100,
    fobj=logloss_objective,
    feval=logloss_metric
)

y_pred = special.expit(model.predict(X_val))

print()
print(f"Test's ROC AUC: {metrics.roc_auc_score(y_val, y_pred):.5f}")
print(f"Test's logloss: {metrics.log_loss(y_val, y_pred):.5f}")


Training until validation scores don't improve for 20 rounds
[100]	fit's auc: 0.73372	val's auc: 0.716931
[200]	fit's auc: 0.742883	val's auc: 0.723078
[300]	fit's auc: 0.750629	val's auc: 0.725809
[400]	fit's auc: 0.756979	val's auc: 0.727177
[500]	fit's auc: 0.762545	val's auc: 0.727823
[600]	fit's auc: 0.767476	val's auc: 0.728189
Early stopping, best iteration is:
[595]	fit's auc: 0.767307	val's auc: 0.728193

Test's ROC AUC: 0.72819
Test's logloss: 0.60909


Training until validation scores don't improve for 20 rounds
[100]	fit's auc: 0.73167	val's auc: 0.714288
[200]	fit's auc: 0.73969	val's auc: 0.719166
[300]	fit's auc: 0.746372	val's auc: 0.721622
[400]	fit's auc: 0.752676	val's auc: 0.722875
[500]	fit's auc: 0.757934	val's auc: 0.723622
[600]	fit's auc: 0.762459	val's auc: 0.723991
[700]	fit's auc: 0.766309	val's auc: 0.724167
Early stopping, best iteration is:
[729]	fit's auc: 0.767411	val's auc: 0.724235

Test's ROC AUC: 0.72424
Test's logloss: 0.61135


In [46]:
val_score = roc_auc_score_at_K(y_pred, y_val)
val_score

0.5370708927937151

0.5370708927937151

In [29]:
model.save_model(f'track_2/model.lgb')

In [ ]:
Local 0.525 LB 0.513
Local 0.538 LB 